In [1]:
# Working notebook
import pandas as pd
import numpy as np
import spectral


In [2]:
# Read the CSV file using HP330 class
hp330 = spectral.HP330()
file_path = 'data/broad.csv'
df = hp330.read_file(file_path)

# Display first few rows
print(f"Total data points: {len(df)}")
print(f"Wavelength range: {df['wavelength'].min()}nm - {df['wavelength'].max()}nm")
print("\nFirst 10 rows:")
df.head(10)


Total data points: 401
Wavelength range: 380nm - 780nm

First 10 rows:


,wavelength,intensity
0,380,0.003049
1,381,0.002975
2,382,0.002944
3,383,0.002810
4,384,0.002723
5,385,0.002642
6,386,0.002547
7,387,0.002535
8,388,0.002538
9,389,0.002508


In [3]:
# Plot the spectral data using Analysis class
analysis = spectral.Analysis()
analysis.plot_spectral(df)


In [4]:
# Calculate attenuation using Filter class
filter_obj = spectral.Filter()
attenuation_df = filter_obj.calculate_attenuation(
    transmitted_file='data/broad-paper.csv',
    incident_file='data/broad.csv'
)

# Display first few rows
print(f"Total data points: {len(attenuation_df)}")
print(f"Wavelength range: {attenuation_df['wavelength'].min()}nm - {attenuation_df['wavelength'].max()}nm")
print("\nFirst 10 rows:")
attenuation_df.head(10)


Total data points: 401
Wavelength range: 380nm - 780nm

First 10 rows:


,wavelength,attenuation
0,380,0.168252
1,381,0.156975
2,382,0.166101
3,383,0.182562
4,384,0.186559
5,385,0.196821
6,386,0.211622
7,387,0.226036
8,388,0.243105
9,389,0.263158


In [5]:
# Plot the attenuation using Analysis class
analysis = spectral.Analysis()
analysis.plot_attenuation(attenuation_df)


In [6]:
# Calculate attenuation for filters 1, 2, and 3
filter_obj = spectral.Filter()
filter_dfs = []

for i in [1, 2, 3]:
    filter_df = filter_obj.calculate_attenuation(
        transmitted_file=f'data/{i}.csv',
        incident_file='data/0.csv'
    )
    filter_dfs.append(filter_df)

# Plot all filters using Analysis class
analysis = spectral.Analysis()
analysis.plot_multiple_attenuation(filter_dfs, labels=['Filter 1', 'Filter 2', 'Filter 3'])


In [7]:
# Calculate and apply Filter 1 to the spectrum
filter_obj = spectral.Filter()
filter_1 = filter_obj.calculate_attenuation(
    transmitted_file='data/1.csv',
    incident_file='data/0.csv'
)

# Apply Filter 1 to the spectrum
analysis = spectral.Analysis()
recovered_spectrum = analysis.remove_filter(df, filter_1)

# Display first few rows
print(f"Total data points: {len(recovered_spectrum)}")
print(f"Wavelength range: {recovered_spectrum['wavelength'].min()}nm - {recovered_spectrum['wavelength'].max()}nm")
print("\nFirst 10 rows of filtered spectrum:")
recovered_spectrum.head(10)


Total data points: 401
Wavelength range: 380nm - 780nm

First 10 rows of filtered spectrum:


,wavelength,intensity
0,380,0.022817
1,381,0.027080
2,382,0.023799
3,383,0.021350
4,384,0.021027
5,385,0.020616
6,386,0.020296
7,387,0.019461
8,388,0.016406
9,389,0.015241


# Blue Light Exposure

In [10]:
# Integrate the recovered spectrum at 415nm with a width of 10nm
# This integrates from 410nm to 420nm (415nm ± 5nm)
analysis = spectral.Analysis()
integrated_value_m2 = analysis.integrate_spectral(
    recovered_spectrum,
    wavelength_start=410,
    wavelength_end=420
)

# Convert from mW/m² to mW/cm² (1 m² = 10,000 cm²)
integrated_value_cm2 = integrated_value_m2 / 10000

print(f"Integrated intensity from 410nm to 420nm: {integrated_value_cm2:.6f} mW/cm²")


Integrated intensity from 410nm to 420nm: 0.000074 mW/cm²


In [11]:
# Calculate exposure time to reach 15 J/cm²
# Energy (J/cm²) = Power (W/cm²) × Time (seconds)
# Time (seconds) = Energy (J/cm²) / Power (W/cm²)

target_energy = 15  # J/cm²
power_w_cm2 = integrated_value_cm2 / 1000  # Convert mW/cm² to W/cm²

exposure_time_seconds = target_energy / power_w_cm2
exposure_time_minutes = exposure_time_seconds / 60

print(f"Power: {integrated_value_cm2:.6f} mW/cm² = {power_w_cm2:.6f} W/cm²")
print(f"Exposure time to reach {target_energy} J/cm²:")
print(f"  {exposure_time_seconds:.2f} seconds")
print(f"  {exposure_time_minutes:.2f} minutes")


Power: 0.000074 mW/cm² = 0.000000 W/cm²
Exposure time to reach 15 J/cm²:
  201895254.55 seconds
  3364920.91 minutes


In [ ]:
# Red Light Exposure

In [12]:
# Integrate the recovered spectrum at 415nm with a width of 10nm
# This integrates from 410nm to 420nm (415nm ± 5nm)
analysis = spectral.Analysis()
integrated_value_m2 = analysis.integrate_spectral(
    recovered_spectrum,
    wavelength_start=630,
    wavelength_end=660
)

# Convert from mW/m² to mW/cm² (1 m² = 10,000 cm²)
integrated_value_cm2 = integrated_value_m2 / 10000

print(f"Integrated intensity from 410nm to 420nm: {integrated_value_cm2:.6f} mW/cm²")


Integrated intensity from 410nm to 420nm: 0.002682 mW/cm²


In [13]:
# Calculate exposure time to reach 15 J/cm²
# Energy (J/cm²) = Power (W/cm²) × Time (seconds)
# Time (seconds) = Energy (J/cm²) / Power (W/cm²)

target_energy = 12  # J/cm²
power_w_cm2 = integrated_value_cm2 / 1000  # Convert mW/cm² to W/cm²

exposure_time_seconds = target_energy / power_w_cm2
exposure_time_minutes = exposure_time_seconds / 60

print(f"Power: {integrated_value_cm2:.6f} mW/cm² = {power_w_cm2:.6f} W/cm²")
print(f"Exposure time to reach {target_energy} J/cm²:")
print(f"  {exposure_time_seconds:.2f} seconds")
print(f"  {exposure_time_minutes:.2f} minutes")


Power: 0.002682 mW/cm² = 0.000003 W/cm²
Exposure time to reach 12 J/cm²:
  4474810.54 seconds
  74580.18 minutes
